In [1]:
import requests
from scrapy import Selector

In [2]:
data=requests.get('https://stats.espncricinfo.com/ci/engine/records/team/series_results.html?class=13;id=2023;type=year')
response=Selector(data)

In [3]:
import csv
filename="2023 ALL MATCHES"
with open(f"{filename}.csv", "w+", newline='') as f:
            writer = csv.writer(f)
            header = ['Team1', 'Team2', 'status','Scorecard_url']
            writer.writerow(header)

In [4]:
series_table=response.xpath('.//caption[contains(text(),"Series results")]/parent::table')
series_urls=series_table.xpath('.//*[contains(@href,"/series/")]/@href').extract()

In [5]:
for series_url in series_urls:
    series_id=series_url.split('/')[-1].replace('.html','')
    series_url=f'https://www.espncricinfo.com/series/big-bash-league-2021-22-{series_id}/match-schedule-fixtures-and-results'
    data3=requests.get(series_url)
    response3=Selector(data3)
    #matchs=response3.xpath('//*[@class="ds-text-compact-xxs" ]')
    matchs=response3.xpath('.//*[contains(@class,"ds-p-4 hover:ds-bg-ui-fill-translucent")]')
    #try:
    for m in matchs:
        teams=m.xpath('.//*[@class="ds-flex ds-items-center ds-min-w-0 ds-mr-1"]/p/text()').extract()
        status=m.xpath('.//*[@class="ds-text-tight-s ds-font-regular ds-line-clamp-2 ds-text-typo"]/span/text()').extract_first()
        scorecard =m.xpath('.//a/@href').extract_first()
        team1=teams[0]
        team2=teams[-1]
        scorecard_url='https://www.espncricinfo.com'+scorecard
        with open(f"{filename}.csv", "a+", newline='') as f:
            writer = csv.writer(f)
            value = [team1,team2,status,scorecard_url]
            writer.writerow(value)